<a href="https://colab.research.google.com/github/willjhliang/traffic-sign-recognition/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Download dataset from github repo
!git clone --quiet https://github.com/willjhliang/traffic-sign-recognition.git
!mv traffic-sign-recognition/* .
!rm -r traffic-sign-recognition

In [ ]:
from copy import deepcopy
import itertools
from tqdm import tqdm

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('seaborn-whitegrid')

from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb

import torch
from torch import Tensor
from torch import nn
from torch import optim
from torch.utils import data
import torchvision

from constants import K, S, class_size, validation_ratio, random_seed
from data import load_data, consolidate_data, split_validation, visualize_data, compare_class_dist
from augment import augment_dataset, visualize_augmentation
from dimensionality_reduction import run_pca, visualize_pca, visualize_pca_per_channel
from torch_utils import load_torch_data, train_model, evaluate_model
from evaluation import generate_confusion_matrix, get_classification_report, generate_class_comparison

# Dataset

In [ ]:
train_data = load_data('data/filtered_images/train')
test_data = load_data('data/filtered_images/test')
labels = pd.read_csv("data/filtered_labels.csv")

In [ ]:
visualize_data(train_data)
compare_class_dist(train_data, test_data)

In [ ]:
X_train, X_train_flattened, y_train = consolidate_data(train_data)
X_test, X_test_flattened, y_test = consolidate_data(test_data)

print(f'X_train shape: {X_train.shape}')
print(f'X_train_flattened shape: {X_train_flattened.shape}')
print(f'X_test shape: {X_test.shape}')

In [ ]:
X_train_pca, X_test_pca, pca = run_pca(X_train_flattened, X_test_flattened)
visualize_pca(pca)

# Models

The following is a set of models we run on the data. Starting with the most simple baseline logistic regression, we move toward more complex models.
1. Logistic Regression
2. K-Nearest Neighbors
3. Adaboost
4. Kernelized SVM
5. Dense Neural Network
6. Convolutional Neural Network
7. Transfer Learning Resnet18, VGG16, EfficientNet, and Vision Transformers

## Logistic Regression

In [ ]:
def logistic_regression(X_train_full, y_train_full, X_test, y_test):
    X_train, X_val, y_train, y_val = split_validation(X_train_full, y_train_full)

    C_values = [0.01, 0.1, 1, 10, 100]
    best_C = -1
    best_acc = 0

    accs = []
    for c in tqdm(C_values, leave=False):
        model = LogisticRegression(
            penalty='l2',
            C=c,
            multi_class = 'multinomial',
            max_iter=1000
        )
        model.fit(X_train, y_train)
        acc = model.score(X_val, y_val)
        accs.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_C = c

    plt.plot(accs)
    plt.xticks(list(range(len(accs))), [str(c) for c in C_values])
    plt.show()
    print(f'Optimal C: {best_C}')

    model = LogisticRegression(
        penalty='l2',
        C=best_C,
        multi_class = 'multinomial',
        max_iter=1000
    )
    model.fit(X_train_full, y_train_full)

    return model.predict(X_test), model.score(X_test, y_test)

## K-Nearest Neighbors

In [ ]:
kf = KFold(n_splits=5)


def evaluate_kfold(model_base, X_train, y_train):
    """Evaluates the given model with K-Fold cross validation."""
    total_acc = 0
    for train_index, val_index in kf.split(X_train): # Iterate through folds
       # Split data into training data and validation data
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
 
        # Train model
        model = clone(model_base)
        model.fit(X_train_fold, y_train_fold)
        total_acc += model.score(X_val_fold, y_val_fold)
       
    avg_acc = total_acc / 5
    return avg_acc


def knn(X_train, y_train, X_test, y_test):
    k_values = [1, 3, 5, 7, 9, 11, 13, 15]
    best_k = -1
    best_acc = 0

    accs = []
    for k_neighbors in tqdm(k_values, leave=False):
        avg_acc = evaluate_kfold(KNeighborsClassifier(n_neighbors = k_neighbors), X_train, y_train)
        accs.append(avg_acc)
        if avg_acc > best_acc:
            best_acc = avg_acc
            best_k = k_neighbors
 
    plt.plot(k_values, accs)
    plt.show()
    print(f"Optimal k: {best_k}")
    
    model = KNeighborsClassifier(n_neighbors=best_k)
    model.fit(X_train, y_train)

    return model.predict(X_test), model.score(X_test, y_test)

## Adaboost

In [ ]:
def adaboost(X_train_full, y_train_full, X_test, y_test):
    X_train, X_val, y_train, y_val = split_validation(X_train_full, y_train_full)

    learning_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
    best_lr = -1
    best_acc = 0

    accs = []
    for lr in tqdm(learning_rates, leave=False):
        model = AdaBoostClassifier(
            DecisionTreeClassifier(max_depth=1),
            n_estimators=200,
            algorithm="SAMME.R",
            learning_rate=lr,
            random_state=random_seed
        )
        model.fit(X_train, y_train)
        acc = model.score(X_val, y_val)
        accs.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_lr = lr
  
    plt.plot(accs)
    plt.xticks(list(range(len(accs))), [str(lr) for lr in learning_rates])
    plt.show()
    print(f'Optimal learning rate: {best_lr}')
    
    model = AdaBoostClassifier(
        DecisionTreeClassifier(max_depth=1),
        n_estimators=200,
        algorithm="SAMME.R",
        learning_rate=lr,
        random_state=random_seed
    )
    model.fit(X_train_full, y_train_full)
 
    return model.predict(X_test), model.score(X_test, y_test)

## XGBoost

In [ ]:
def xgboost(X_train_full, y_train_full, X_test, y_test):
    X_train, X_val, y_train, y_val = split_validation(X_train_full, y_train_full)

    learning_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
    best_lr = -1
    best_acc = 0
 
    accs = []
    for lr in tqdm(learning_rates, leave=False):
        model = xgb.XGBClassifier(n_estimators=200, max_depth=1, learning_rate=0.1, objective='multi:softmax', booster='gbtree', num_classes=K)
        model.fit(X_train, y_train)
        acc = model.score(X_val, y_val)
        accs.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_lr = lr
     
    plt.plot(accs)
    plt.xticks(list(range(len(accs))), [str(lr) for lr in learning_rates])
    plt.show()
    print(f'Optimal learning rate: {best_lr}')
  
    model = xgb.XGBClassifier(n_estimators=200, max_depth=1, learning_rate=lr, objective='multi:softmax', booster='gbtree', num_classes=K)
    model.fit(X_train_full, y_train_full)

    return model.predict(X_test), model.score(X_test, y_test)


## Kernelized SVM

In [ ]:
def kernel_svm(X_train_full, y_train_full, X_test, y_test):
    X_train, X_val, y_train, y_val = split_validation(X_train_full, y_train_full)

    kernels = ['linear', 'poly', 'rbf']
    C_values = [0.01, 0.1, 1, 10, 100]
    best_kernel = ''
    best_C = -1
    best_acc = 0

    accs = {}
    for kernel in kernels:
        accs[kernel] = []
    for kernel, c in tqdm(itertools.product(kernels, C_values), leave=False):
        model = SVC(kernel=kernel, C=c)
        model.fit(X_train, y_train)
        acc = model.score(X_val, y_val)
        accs[kernel].append(acc)
        if acc > best_acc:
            best_acc = acc
            best_C = c
            best_kernel = kernel
    
    best_accs = [max(accs[kernel]) for kernel in kernels]
    plt.bar(kernels, best_accs)
    plt.xticks(list(range(len(best_accs))), kernels)
    plt.show()
    print(f'Optimal kernel: {best_kernel}')

    model = SVC(kernel=best_kernel, C=best_C)
    model.fit(X_train_full, y_train_full)

    return model.predict(X_test), model.score(X_test, y_test)

## Dense Neural Network

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(32 * 32 * 3, 128)
        self.layer2 = nn.Linear(128, 64)
        self.out_layer = nn.Linear(64, K)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.out_layer(x)
        return x

## Convolutional Neural Network

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.batch_norm_1 = nn.BatchNorm2d(32)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.batch_norm_2 = nn.BatchNorm2d(32)
        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_3 = nn.BatchNorm2d(64)
        self.conv_4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_4 = nn.BatchNorm2d(64)
        self.dropout_1 = nn.Dropout(0.5)
        self.conv_5 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_5 = nn.BatchNorm2d(64)
        self.conv_6 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_6 = nn.BatchNorm2d(64)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_1 = nn.Linear(4 * 4 * 64, 128)
        self.dropout_3 = nn.Dropout(0.25)
        self.linear_2 = nn.Linear(128, K)

    def forward(self, x):
        x = self.relu(self.batch_norm_1(self.conv_1(x)))
        x = self.relu(self.batch_norm_2(self.conv_2(x)))
        x = self.max_pool2d(x)
        x = self.relu(self.batch_norm_3(self.conv_3(x)))
        x = self.relu(self.batch_norm_4(self.conv_4(x)))
        x = self.dropout_1(x)
        x = self.max_pool2d(x)
        x = self.relu(self.batch_norm_5(self.conv_5(x)))
        x = self.relu(self.batch_norm_6(self.conv_6(x)))
        x = self.dropout_2(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_3(x)
        x = self.linear_2(x)
        return x

# Transfer Learning

In [ ]:
def Resnet():
    model = torchvision.models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, K)
    return model

In [ ]:
def VGG16():
    model = torchvision.models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, K)
    return model

In [ ]:
def EfficientNet():
    model = torchvision.models.efficientnet_b0(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, K)
    return model

In [ ]:
class UpscaleDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, transform):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.numpy()
        img = self.transform(torch.from_numpy(self.X[idx].astype(np.float32)))
        label = torch.tensor(self.y[idx])

        return img, label


def load_vit_data(X_train, y_train, X_test, y_test):
    X_train, X_val, y_train, y_val = split_validation(X_train, y_train)
    transform = torchvision.transforms.Resize((224, 224))
    train_set = UpscaleDataset(X_train, y_train, transform)
    val_set = UpscaleDataset(X_val, y_val, transform)
    test_set = UpscaleDataset(X_test, y_test, transform)
    train_loader = data.DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = data.DataLoader(val_set, batch_size=32, shuffle=True)
    test_loader = data.DataLoader(test_set, batch_size=32, shuffle=True)
    return train_loader, val_loader, test_loader


def VisionTransformer():
    model = torchvision.models.vit_b_16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.heads[-1].in_features
    model.heads[-1] = nn.Linear(num_ftrs, K)
    return model

# Training and Evaluation
We now run all the models defined above.

In [ ]:
print('========== Logistic Regression ==========')
y_pred_log, acc = logistic_regression(X_train_flattened, y_train, X_test_flattened, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
log_report, log_df = get_classification_report(y_test, y_pred_log, labels)
generate_class_comparison(log_report, 'f1-score', labels)

In [ ]:
print('========== Logistic Regression (PCA) ==========')
y_pred_log, acc = logistic_regression(X_train_pca, y_train, X_test_pca, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
log_report, log_df = get_classification_report(y_test, y_pred_log, labels)
generate_class_comparison(log_report, 'f1-score', labels)

In [ ]:
print('========== KNN ==========')
y_pred_knn, acc = knn(X_train_flattened, y_train, X_test_flattened, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
knn_reportdict, knn_df = get_classification_report(y_test, y_pred_knn, labels)
generate_class_comparison(knn_reportdict, 'f1-score', labels)

In [ ]:
print('========== KNN (PCA) ==========')
y_pred_knnpca, acc = knn(X_train_pca, y_train, X_test_pca, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
knnpca_reportdict, knnpca_df = get_classification_report(y_test, y_pred_knnpca, labels)
generate_class_comparison(knnpca_reportdict, 'f1-score', labels)

In [ ]:
print('========== Adaboost (PCA) ==========')
y_pred_ada, acc = adaboost(X_train_pca, y_train, X_test_pca, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
ada_reportdict, ada_df = get_classification_report(y_test, y_pred_ada, labels)
generate_class_comparison(ada_reportdict, 'f1-score', labels)

In [ ]:
print('========== XGBoost (PCA) ==========')
y_pred_xgb, acc = xgboost(X_train_pca, y_train, X_test_pca, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
xgb_reportdict, xbg_df = get_classification_report(y_test, y_pred_xgb, labels)
generate_class_comparison(xgb_reportdict, 'f1-score', labels)

In [ ]:
print('========== Kernelized SVM ==========')
y_pred_svm, acc = kernel_svm(X_train_flattened, y_train, X_test_flattened, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
svm_reportdict, svm_df = get_classification_report(y_test, y_pred_svm, labels)
generate_class_comparison(svm_reportdict, 'f1-score', labels)

In [ ]:
print('========== Kernelized SVM (PCA) ==========')
y_pred_svm, acc = kernel_svm(X_train_pca, y_train, X_test_pca, y_test)
print(f'Test Accuracy: {acc}')

In [ ]:
svm_reportdict, svm_df = get_classification_report(y_test, y_pred_svm, labels)
generate_class_comparison(svm_reportdict, 'f1-score', labels)